In [1]:
import pandas as pd
import numpy as np
from itertools import permutations, product, combinations

In [3]:
df_mod = pd.read_csv('mod.csv')
df_mod.columns
mod_mass_list = pd.unique(df_mod['Exact Mass'])
df_mod.rename(columns={'Exact Mass': 'Mass'}, inplace=True)

In [4]:
def ppm2dm(m, ppm=10):
    return (m*ppm)/1E6

In [22]:
def get_permutations(m, n):
    l = list(product(range(m+1), repeat=n))
    l = [i for i in l if sum(i) in range(m-3, m+2) and i[2] > 0]
    l = np.array(l)
    return l

In [23]:
def handle_4bases(mass, masses=[329.0525, 305.0413, 345.0474, 306.0253]):
    size = int(mass/320)
    perms = get_permutations(size, len(masses))
    seq_masses = np.matmul(perms, masses)
    seq_masses_pd = pd.Series(seq_masses)
    res_masses = perms.copy()
    res_masses = pd.DataFrame(res_masses)
#     print(res_masses.index)
#     print(seq_masses_pd.index)
    res_masses['Mass'] = seq_masses_pd
    return res_masses[res_masses.Mass <= mass+1]
#     return seq_masses[seq_masses <= mass+1]

In [24]:
def find_mass_in_df(mass, df, ppm=10):
    dm = ppm2dm(mass, ppm)
    low, high = mass - dm, mass + dm
#     print("low {} high {}".format(low, high))
    return df[(df.Mass>low) & (df.Mass<high)]
#     mask = np.ma.masked_inside(ds, low, high).mask
#     return ds[mask]

In [25]:
dfm_rows = list(dfm.iterrows())
mass_cb = list()
mass_cb.extend(list(combinations(dfm_rows, 1)))
mass_cb.extend(list(combinations(dfm_rows, 2)))
mass_cb.extend(list(combinations(dfm_rows, 3)))

In [26]:
mass_cb_sum = [(sum([item[1].Mass for item in row]), [item[0] for item in row]) for row in mass_cb]
mass_cb_sum.sort()
mass_cb_sum[:4]

[(308.0409525, [4]), (319.0569369, [2]), (320.0409525, [3]), (330.03654, [12])]

In [28]:
# mass = 2567.37756
mass = 4963.6122
# mass = 9183.4049
# mass = 13395.9393
seq_masses = handle_4bases(mass)
len(seq_masses)

diff = seq_masses
diff['Mass'] = mass - seq_masses['Mass']
# diff = pd.Series(diff)
# diff = diff[(diff.Mass > df_mod.Mass.min()) & (diff.Mass < df_mod.Mass.max())]
valid_diff = diff[diff.Mass > df_mod.Mass.min()]
# valid_diff.sort()
# valid_diff

count = 0
full_msg = ""
for idx, item in enumerate(mass_cb_sum):
    found = find_mass_in_df(item[0], valid_diff, 25)
    valid_found = found.dropna()
    if valid_found.size > 0:
#         print("found {} \n valid_found--->\n{}\n<---".format(mass_cb_sum[idx], valid_found.iloc[:, 0:4]))
        index = mass_cb_sum[idx][1]
#         print(list(dfm.loc[index]['Symbol']))
        
        for idx, row in valid_found.iterrows():
            msg = "{}A {}C {}G {}U ".format(int(row[0]), int(row[1]), int(row[2]), int(row[3])) 
            msg += "{}\n".format(' '.join(list(dfm.loc[index]['Symbol'])))
            full_msg += msg
            count += 1

print("found {} possibilities\n".format(count))
print(full_msg)

found 43 possibilities

1A 8C 1G 5U m3C
1A 9C 1G 4U m5U
0A 9C 1G 5U m1A
0A 10C 1G 4U m1I
1A 0C 8G 5U m1I
0A 9C 1G 4U m3C I
0A 10C 1G 3U m5U I
1A 0C 8G 4U m5U I
0A 0C 8G 5U m1A I
0A 1C 8G 4U m1I I
1A 0C 7G 5U m1G I
0A 0C 6G 7U hU ms2i6A
0A 0C 5G 8U ac4C ms2i6A
1A 2C 3G 7U m22G t6A
2A 0C 6G 4U ac4C m5U m1I
1A 0C 6G 5U ac4C m1A m1I
2A 0C 5G 5U m1G ac4C m1I
2A 0C 5G 5U ac4C m22G I
1A 0C 4G 7U hU t6A I
1A 1C 4G 6U m3C m5U t6A
2A 1C 3G 6U ac4C hU t6A
0A 0C 4G 8U m3C m22G ms2i6A
0A 1C 4G 7U m3C m1A t6A
0A 1C 4G 7U m5U m22G ms2i6A
0A 2C 4G 6U m3C t6A m1I
0A 2C 4G 6U m5U m1A t6A
0A 3C 4G 5U m5U t6A m1I
1A 1C 3G 7U m1G m3C t6A
1A 2C 3G 6U m1G m5U t6A
2A 0C 2G 8U m5U m22G t6A
0A 1C 3G 8U m1G m22G ms2i6A
0A 2C 3G 7U m1G m1A t6A
0A 3C 3G 6U m1G t6A m1I
0A 3C 3G 6U m22G t6A I
1A 0C 2G 9U m22G m1A t6A
1A 0C 2G 9U m3C i6A t6A
1A 1C 2G 8U m22G t6A m1I
1A 1C 2G 8U m5U i6A t6A
2A 0C 1G 9U m1G m22G t6A
2A 2C 1G 7U hU yW I
0A 1C 2G 9U i6A m1A t6A
0A 2C 2G 8U i6A t6A m1I
1A 1C 1G 9U m1G i6A t6A

